# Find combinations of images

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('mapping.csv')

In [9]:
df.head(5)

,folder,file name,smoke
0,20170625-BBM-bm-n-mobo,1498413993_-02400.jpg,0
1,20170625-BBM-bm-n-mobo,1498414053_-02340.jpg,0
2,20170625-BBM-bm-n-mobo,1498414113_-02280.jpg,0
3,20170625-BBM-bm-n-mobo,1498414173_-02220.jpg,0
4,20170625-BBM-bm-n-mobo,1498414233_-02160.jpg,0


In [3]:
def apply_rules(smoke1, smoke2):
    if smoke1 == 0 and smoke2 == 0:
        return 0
    elif smoke1 == 0 and smoke2 == 1:
        return 1
    elif smoke1 == 1 and smoke2 == 1:
        return 1

In [4]:
def find_combinations_one_camera(df, file_name_col, smoke_col):
    new = []
    for i in range(0, len(df[file_name_col])):
        for j in range(0, len(df[file_name_col])):
            new.append(
            {
                "image1": df[file_name_col][i],
                "image2": df[file_name_col][j],
                "label": apply_rules(df[smoke_col][i], df[smoke_col][j])
            })
    return pd.DataFrame(new)

In [7]:
camera = df.groupby('folder')['file name'].count().reset_index(name='images')
camera

,folder,images
0,20170625-BBM-bm-n-mobo,81
1,20170708-Whittier-syp-n-mobo-c,71
2,20171207-Lilac-rm-s-mobo,71
3,20180611-fallbrook-rm-w-mobo-c,77
4,20180614-Bridle-hp-n-mobo-c,81
5,20180614-Hope-wc-e-mobo-c,64
6,20180704-Benton-hp-n-mobo-c,61
7,20180706-West-lp-n-mobo-c,81
8,20180725-Cranston-hp-n-mobo-c,67
9,20190529-94Fire-lp-s-mobo-c,64


In [8]:
final = pd.DataFrame()
for i in range(0, len(camera['folder'])):
    filtered = df[df['folder'] == camera['folder'][i]].reset_index()
    combinations = find_combinations_one_camera(filtered, 'file name', 'smoke')
    final = pd.concat([final, combinations], ignore_index=True)

In [10]:
final.isnull().sum()

image1        0
image2        0
label     23043
dtype: int64

What should we do with the combination smoke(t) and no_smoke(t-1) ?  
We have 23043 labels with this case.

For now, I remove these cases from our database.

In [11]:
final.dropna(inplace=True)

In [42]:
final.isnull().sum()

image1    0
image2    0
label     0
dtype: int64

In [43]:
final.groupby('label').count()

,image1,image2
label,,
0,177421,177421
1,53606,53606


In [17]:
final['label'] = final['label'].astype('int32')

In [30]:
final.reset_index(inplace=True)

In [40]:
final

,image1,image2,label
0,1498413993_-02400.jpg,1498413993_-02400.jpg,0
1,1498413993_-02400.jpg,1498414053_-02340.jpg,0
2,1498413993_-02400.jpg,1498414113_-02280.jpg,0
3,1498413993_-02400.jpg,1498414173_-02220.jpg,0
4,1498413993_-02400.jpg,1498414233_-02160.jpg,0
...,...,...,...
231022,1572721175.jpg,1572720935.jpg,0
231023,1572721175.jpg,1572720995.jpg,0
231024,1572721175.jpg,1572721055.jpg,0
231025,1572721175.jpg,1572721115.jpg,0


In [41]:
final.to_csv('combinations.csv')